# CPS Report

The following notebook requires Ubuntu 22.04 LTS (Jammy Jellyfish) to be executed

### Preliminary installations with ROS 2 Humble/Ubuntu 22.04 

https://docs.ros.org/en/humble/Tutorials.html

https://docs.ros.org/en/humble/Installation/Ubuntu-Install-Debians.html

In [ ]:
!sudo apt install software-properties-common
!sudo add-apt-repository universe

In [ ]:
!sudo curl -sSL https://raw.githubusercontent.com/ros/rosdistro/master/ros.key -o /usr/share/keyrings/ros-archive-keyring.gpg
!echo "deb [arch=$(dpkg --print-architecture) signed-by=/usr/share/keyrings/ros-archive-keyring.gpg] http://packages.ros.org/ros2/ubuntu $(. /etc/os-release && echo $UBUNTU_CODENAME) main" | sudo tee /etc/apt/sources.list.d/ros2.list > /dev/null

In [ ]:
# ROS installation
!sudo apt update
!sudo apt upgrade
!sudo apt install ros-humble-desktop

!sudo apt install ros-dev-tools

# Gazebo installation
!sudo apt install libgazebo-dev
!sudo apt install ros-humble-gazebo-ros-pkgs

In [ ]:
# ALWAYS REMEMBER TO SOURCE THIS FILE (or add it in the .bashrc)
!source /opt/ros/humble/setup.bash

!echo "source /opt/ros/humble/setup.bash" >> ~/.bashrc

https://docs.px4.io/main/en/dev_setup/dev_env_linux_ubuntu.html#ros-2

In [ ]:
# download PX4
!git clone https://github.com/PX4/PX4-Autopilot.git --recursive

# install
!bash ./PX4-Autopilot/Tools/setup/ubuntu.sh -no-sim-tools --no-nuttx

# then restart computer

# additional dependencies
!sudo apt-get install protobuf-compiler libeigen3-dev libopencv-dev -y

### .SDF to .URDF confersion of the iris model

https://github.com/andreasBihlmaier/pysdf __ NOT WORKING

In [ ]:
!git clone https://github.com/andreasBihlmaier/pysdf
!pip install python-sdformat

# Conversion example
!cd ROS2_ws/src/pysdf/scripts
!python3 sdf2urdf.py ~/PX4-Autopilot/Tools/simulation/gazebo-classic/sitl_gazebo-classic/models/iris/iris.sdf ~/PX4-Autopilot/Tools/simulation/gazebo-classic/sitl_gazebo-classic/models/iris/iris.urdf

https://github.com/theg4sh/sdf2urdf

In [ ]:
!git clone https://github.com/theg4sh/sdf2urdf.git
!sudo pip3 install xacro python-dotenv pyparsing

!cd ROS_ws/src/sdf2urdf
!export PYTHON_PATH=$(pwd)/GazeboMaterial
# Conversion example
!./sdf2urdf.py ~/PX4-Autopilot/Tools/simulation/gazebo-classic/sitl_gazebo-classic/models/iris/iris.sdf

### Generic PX4 Gazebo simulation with IRIS drone

In [ ]:
!cd PX4-Autopilot
!make px4_sitl gazebo-classic